In [1]:
import pandas as pd
import numpy as np
import csv
import re
import string
from itertools import product

In [2]:
test_file_plate_2_1 = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\Protein titration LissamineFA_22267.csv"
test_file_plate_2_2 = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\Protein titration LissamineFA_22741.csv"
test_file_list_A = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\Outputfiles\\csv\\listA.csv"
test_file_repeat_read_plate_1 = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\Outputfiles\\csv\\repeat read_plate1.csv"

In [3]:
reader = csv.reader(open(test_file_repeat_read_plate_1, 'r'), delimiter=',') 

wells = 384
all_data = [] 
data_frames = {}
counter = 1

# Create a list of lists with all data in the csv file
for row in reader:
    all_data.append(row)

total_lines = len(all_data)

# Find the data        
for index, item in enumerate(all_data):
    if item != [] and re.findall(r"Plate information", item[0]) == ['Plate information'] and re.findall(r"Formula", all_data[index+1][10]) != ['Formula']:
        skiprows = index + 9   # Set the skiprows parameter
        skiprows_meta = index + 1
        skipfooter_meta = total_lines - (index + 4)
        
        end_of_data = 0
        # find the index of the first blank row after the data table
        while end_of_data == 0:   
            for index, item in enumerate(all_data):
                if item == [] and index > skiprows:  
                    end_of_data = index
                    break
                    
        skipfooter = total_lines - end_of_data   # calucalte the skipfooter parameter
        
        col_names = list(np.arange(1,int((wells/16)+1)))   # define column names so that the last column is not imported
        
        # extract data into pandas df
        data = pd.read_csv(test_file_repeat_read_plate_1, sep=',', names=col_names, index_col=False, engine='python', skiprows=skiprows, skipfooter=skipfooter, encoding='utf-8')
        # extract metadata into pandas df
        metadata = pd.read_csv(test_file_repeat_read_plate_1, sep=',', engine='python', skiprows=skiprows_meta, skipfooter=skipfooter_meta, encoding='utf-8')
        
        dimension = data.shape   # get the dimension of the data table as a tuple
        letters = list(string.ascii_uppercase)[0:dimension[0]]   # generate letters for the new data table
        col_nums = list(np.arange(1, dimension[1]+1).astype(str))   # generate numbers for the new data table
        well_ids = ['%s%s' % (item[0], item[1]) for item in product(letters, col_nums)]   # generate well IDs for the new data table
        
        data_array = np.reshape(data.to_numpy(), (wells, 1))
        #new_data = pd.DataFrame(data=data_array, index=well_ids, columns=['p'])
        
        if counter % 1 == 0:
            new_data = pd.DataFrame(data=data_array, index=well_ids, columns=['p'])
            data_frames[f'repeat_{int(counter)}'] = {'metadata':{'p': metadata, 's': ''}, 'data': {'p': new_data, 's':''}}
        elif counter % 1 != 0:
            new_data = pd.DataFrame(data=data_array, index=well_ids, columns=['s'])
            data_frames[f'repeat_{int(counter-0.5)}']['metadata']['s'] = metadata
            data_frames[f'repeat_{int(counter-0.5)}']['data']['s'] = new_data   
        
        counter = counter + 0.5

In [4]:
len(data_frames)

2

In [5]:
data_frames['repeat_1']['metadata']['p']

,Plate,Repeat,Barcode,Measured height,Chamber temperature at start,Chamber temperature at end,Humidity at start,Humidity at end,Ambient temperature at start,Ambient temperature at end,Group,Label,ScanX,ScanY,Measinfo,Kinetics,Measurement date,Unnamed: 17
0,1,1,NaN,14.4,18.98,18.8,61.7,61.5,18.98,18.9,1,Copy of Kris FP Fluorescein anisotropy(1),0,0,De=1st Ex=Top Em=Top Wdw=N/A (15),0,17/11/2020 13:33:43,NaN


In [6]:
data_frames['repeat_2']['metadata']['s']

,Plate,Repeat,Barcode,Measured height,Chamber temperature at start,Chamber temperature at end,Humidity at start,Humidity at end,Ambient temperature at start,Ambient temperature at end,Group,Label,ScanX,ScanY,Measinfo,Kinetics,Measurement date,Unnamed: 17
0,1,2,NaN,14.4,18.8,18.7,61.5,61.8,18.9,19.1,1,Copy of Kris FP Fluorescein anisotropy(1),0,0,De=2nd Ex=Top Em=Top Wdw=N/A (142),0,17/11/2020 13:35:26,NaN


In [7]:
data_frames['repeat_1']['data']['s']

,s
A1,20469296.0
A2,29296716.0
A3,18210982.0
A4,23159988.0
A5,24960618.0
...,...
P20,NaN
P21,NaN
P22,NaN
P23,NaN


In [8]:
data_frames['repeat_2']['data']['p']

,p
A1,18964719.0
A2,27435568.0
A3,16237095.0
A4,21291729.0
A5,22591167.0
...,...
P20,NaN
P21,NaN
P22,NaN
P23,NaN
